<h3>IMPORTS</h3>

In [107]:
#IMPORTS
import pandas as pd
import sqlite3
import os


<h3>METHODS</h3>

In [ ]:
#process 1 csv file 

def extract_data(file_path):

    df = pd.read_csv(file_path, encoding='latin1', header = None, keep_default_na=False, index_col=None)
    df = df.drop([0,1,2,3,4,5,6])  # Drop the first 7 rows

    try:
        df = df.iloc[:, [0, 2]]

        df = df[['  Station Name ', '  Date ',
        ' Rain 0900-0900 (mm)', 
        ' Maximum Temperature (°C)', ' Minimum Temperature (°C)',
        'Maximum Relative Humidity (%)', 'Minimum Relative Humidity (%)',
        'Average 10m Wind Speed (m/sec)']] #keep only required columns

    except:
        raise Exception("column names not properly formatted in file")

    row1 = df.iloc[0]
    row2 = df.iloc[1]
    row3 = df.iloc[2]
    row4 = df.iloc[3]


    def combine_rows(row1, row2):
        combined_row = {}
        for col in row1.index:
            combined_row[col] = str(row1[col]) + " " + str(row2[col])
        return pd.DataFrame([combined_row]).iloc[0]  
        

    header_row = combine_rows(combine_rows(combine_rows(row1, row2), row3), row4)

    # #Set top row as header row values
    df.columns = header_row.tolist()
    df = df.drop(df.index[[0,1,2,3]])  # Drop the first four rows which are now redundant

    try:
        df = df[['  Station Name ', '  Date ',
        ' Rain 0900-0900 (mm)', 
        ' Maximum Temperature (°C)', ' Minimum Temperature (°C)',
        'Maximum Relative Humidity (%)', 'Minimum Relative Humidity (%)',
        'Average 10m Wind Speed (m/sec)']] #keep only required columns

    except:
        raise Exception("column names not properly formatted in file")

    df = df.reset_index(drop=True)  # Reset the index

    return df


print(extract_data("./Data/vic/aireys_inlet-201508.csv"))

     Station Name        Date   Rain 0900-0900 (mm)  Maximum Temperature (°C)  \
0     AIREYS INLET  01/08/2015                  1.0                      11.8   
1     AIREYS INLET  02/08/2015                  0.0                      12.9   
2     AIREYS INLET  03/08/2015                  3.6                       9.9   
3     AIREYS INLET  04/08/2015                  0.2                      11.5   
4     AIREYS INLET  05/08/2015                  7.2                      10.3   
5     AIREYS INLET  06/08/2015                  1.6                      13.4   
6     AIREYS INLET  07/08/2015                  0.8                      13.5   
7     AIREYS INLET  08/08/2015                  0.6                      13.7   
8     AIREYS INLET  09/08/2015                  0.0                      14.5   
9     AIREYS INLET  10/08/2015                  1.0                      14.8   
10    AIREYS INLET  11/08/2015                  1.0                      14.7   
11    AIREYS INLET  12/08/20

In [131]:
def insert_data_from_file(file_path, connection, db_table):
    try:
        df = extract_data(file_path)
    except Exception as e:
        return e

    try:
        df.to_sql(db_table, connection, if_exists='replace', index=False)   
    except Exception as e:
        return e

<h3>Establish file paths & connections</h3>

In [132]:
# File paths
data_folder_path = "./Data/"
db_file_path = "./WeatherStationDB.db"

db_conn = sqlite3.connect(db_file_path)
db_table = 'weather_data'


<h3>INSERT VIC data</h3>

In [139]:
#extract data from files in folder and insert into db

# Get the list of all files in directory
state = "vic"
state_data_path = data_folder_path + state
dir_list = os.listdir(state_data_path)


for data_file in dir_list:
    print(data_file)
    #get relative data file path
    data_file_path = state_data_path + "/" + data_file

    df = extract_data(data_file_path)

    # 'if_exists' can be 'fail', 'replace', or 'append'
    # 'index=False' prevents writing the DataFrame index as a column in the database
    df.to_sql(db_table, db_conn, if_exists='replace', index=False)
    
    """
    # The 'axis=0' argument specifies concatenation along rows (vertical).
    # 'ignore_index=True' resets the index of the resulting DataFrame for a continuous index.
    state_weather_data_df = pd.concat([state_weather_data_df, df], axis=0, ignore_index=True)
    """




    

aireys_inlet-201508.csv


Exception: column names not properly formatted in file